In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
import plotnine as p9


In [2]:
X_train = pd.read_csv("./data/external/train_logs.csv")

X_train = (
    X_train
    .sort_values(["id", "event_id"], ascending=[True, True])
    )

In [4]:
PUNCTUATION = X_train.loc[(
    (X_train['activity'] == 'Input')
    & (~ X_train['text_change'].isin(['q', ' ']))
    ), 'text_change'].unique()

In [7]:
# no explicit record for a pause. pauses are omitted.
PAUSE_THRESHOLD_MS = 1000

X_train['up_time_lag1'] = (
    X_train
    .groupby(['id'])
    ['up_time']
    .shift(1)
    )
X_train['preceding_pause_time'] = (
    X_train['down_time'] - X_train['up_time_lag1']
    )
# expect some negative pause times -- interpret as, no real pause
has_no_real_pause = X_train['preceding_pause_time'] <= PAUSE_THRESHOLD_MS
X_train.loc[has_no_real_pause, 'preceding_pause_time'] = None

# summarize pause distr
MS_IN_PAUSE_BUCKET_MAX = 200e3
PAUSE_BUCKET_STEP_MS = 500

X_train['preceding_pause_time_bucket'] = pd.cut(
    X_train['preceding_pause_time'],
    bins=np.arange(
        0, 
        MS_IN_PAUSE_BUCKET_MAX,
        PAUSE_BUCKET_STEP_MS
        )
    )

X_train['preceding_pause_time_bucket'].value_counts()

# WARNING: this representation of pause distribution is dense & large
# a few parameters from distribution model far more succinct

preceding_pause_time_bucket
(1000.0, 1500.0]        126117
(1500.0, 2000.0]         62487
(2000.0, 2500.0]         37948
(2500.0, 3000.0]         25823
(3000.0, 3500.0]         18885
                         ...  
(194000.0, 194500.0]         0
(184500.0, 185000.0]         0
(175000.0, 175500.0]         0
(500.0, 1000.0]              0
(0.0, 500.0]                 0
Name: count, Length: 399, dtype: int64

In [8]:
# if pause exceeds threshold duration, a "burst" has ended
MS_PER_S = 1000
SECONDS_PER_BURST = 2

X_train['is_new_burst_start'] = (
    X_train['preceding_pause_time'] > MS_PER_S * SECONDS_PER_BURST
    ).astype(int)
X_train['is_new_burst_start'][0] = 1
X_train['burst_id'] = (
    X_train
    .groupby(['id'])
    ['is_new_burst_start']
    .cumsum()
    )
X_train['burst_time_start'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['down_time']
    .transform('min')
    )
X_train['burst_time_end'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['up_time']
    .transform('max')
    )
X_train['burst_duration'] = X_train['burst_time_end'] - X_train['burst_time_start']

C:\Users\alexa\AppData\Local\Temp\ipykernel_6528\3022411987.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
# one-way cursor movement might be most productive
# jumping around is choppy
X_train['cursor_position_lag1'] = (
    X_train
    .groupby(['id'])
    ['cursor_position']
    .shift(1)
    )

X_train['has_cursor_position_moved_right'] = (
    X_train['cursor_position'] > X_train['cursor_position_lag1']
    ).astype(int)

# farthest position cursor has _edited_, with recorded input
X_train['cursor_position_cummax'] = (
    X_train
    .groupby(['id'])
    ['cursor_position']
    .cummax()
    )
X_train.loc[X_train['activity'] != 'Input', 'cursor_position_cummax'] = None
X_train['cursor_position_cummax'] = (
    X_train
    .groupby(['id'])
    ['cursor_position_cummax']
    .ffill()
    )

X_train['cursor_position_vs_max'] = (
    X_train['cursor_position'] - X_train['cursor_position_cummax']
    )

In [18]:
# word count offers a productivity measure
X_train['word_count_lag1'] = (
    X_train
    .groupby(['id'])
    ['word_count']
    .shift(1)
    )

X_train['word_count_delta_event'] = (
    X_train['word_count'] - X_train['word_count_lag1']
    )

X_train['word_count_delta_burst'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['word_count_delta_event']
    .transform('sum')
    )

# word length offers a content quality measure.
# hard to track entire words sequence in rolling fashion.
    # every word's length, in a list of one element per word?  
# more tractable to track very latest string
is_edit_to_max_string = X_train['cursor_position_vs_max'] == 0
X_train['is_new_max_string_start'] = (
    is_edit_to_max_string
    & (X_train['activity'] == "Input")
    & (X_train["text_change"] == ' ')
    )
X_train['n_alphanum_char_added_to_max_string'] = 0
is_alphanumeric_addition = (
    (X_train['activity'] == "Input")
    & (X_train["text_change"] == 'q')
    )
X_train.loc[
    (is_alphanumeric_addition & is_edit_to_max_string), 
    'n_alphanum_char_added_to_max_string'
    ] = 1
is_alphanumeric_subtraction = (
    (X_train['activity'] == "Remove/Cut")
    & (X_train['up_event'] == 'Backspace')
    & (X_train["text_change"] == 'q')
    )
X_train.loc[
    (is_alphanumeric_subtraction & is_edit_to_max_string), 
    'n_alphanum_char_added_to_max_string'
    ] = -1
# example: 2nd string, 2 characters in.
# considering cumsum for each character in 2nd max string, 
# subtract those characters from 1st
X_train['length_max_string'] = (
    X_train
    .groupby(['id'])
    ['n_alphanum_char_added_to_max_string']
    .cumsum() 
    ) - (
    X_train
    .groupby(['id'])
    ['n_alphanum_char_added_to_max_string']
    .cumsum()
    .where(X_train['is_new_max_string_start']) 
    .ffill()
    .fillna(0)
    )

In [19]:
X_train.head(20)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,...,cursor_position_lag1,has_cursor_position_moved_right,cursor_position_cummax,cursor_position_vs_max,word_count_lag1,word_count_delta_event,word_count_delta_burst,is_new_max_string_start,n_alphanum_char_added_to_max_string,length_max_string
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,False,0,0.0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,...,0.0,0,NaN,NaN,0.0,0.0,0.0,False,0,0.0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,...,0.0,0,NaN,NaN,0.0,0.0,12.0,False,0,0.0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,...,0.0,1,1.0,0.0,0.0,1.0,12.0,False,1,1.0
4,001519c8,5,107196,107323,127,Input,q,q,q,2,...,1.0,1,2.0,0.0,1.0,0.0,12.0,False,1,2.0
5,001519c8,6,107296,107400,104,Input,q,q,q,3,...,2.0,1,3.0,0.0,1.0,0.0,12.0,False,1,3.0
6,001519c8,7,107469,107596,127,Input,q,q,q,4,...,3.0,1,4.0,0.0,1.0,0.0,12.0,False,1,4.0
7,001519c8,8,107659,107766,107,Input,q,q,q,5,...,4.0,1,5.0,0.0,1.0,0.0,12.0,False,1,5.0
8,001519c8,9,107743,107852,109,Input,q,q,q,6,...,5.0,1,6.0,0.0,1.0,0.0,12.0,False,1,6.0
9,001519c8,10,107840,107978,138,Input,Space,Space,,7,...,6.0,1,7.0,0.0,1.0,0.0,12.0,True,0,0.0


In [ ]:
# if thoughts aren't separated by punctuation, writing won't score well
X_train['is_thought_delimiting_punctuation'] = (
    (X_train['text_change'] == ".")
    | (X_train['text_change'] == ". ")
    | (X_train['text_change'] == ",")
    | (X_train['text_change'] == "-")
    | (X_train['text_change'] == "!")
    | (X_train['text_change'] == ";")
    | (X_train['text_change'] == "?")
    | (X_train['text_change'] == ":")
    ).astype(int)

X_train['is_special_punctuation'] = (
    (X_train['text_change'] == "=")
    | (X_train['text_change'] == "/")
    | (X_train['text_change'] == "\\")
    | (X_train['text_change'] == "(")
    | (X_train['text_change'] == ")")
    | (X_train['text_change'] == "\n")
    | (X_train['text_change'] == "[")
    | (X_train['text_change'] == "]")
    | (X_train['text_change'] == ">")
    | (X_train['text_change'] == "<")
    | (X_train['text_change'] == "$")
    | (X_train['text_change'] == "*")
    | (X_train['text_change'] == "&")
)

In [ ]:
# windows allow for time-sequence features
TOTAL_MIN = 30
SECONDS_PER_MIN = 60
SECONDS_PER_WINDOW = 30

X_train['window_30s'] = pd.cut(
    X_train['down_time'],
    bins=np.arange(
        0, 
        TOTAL_MIN * SECONDS_PER_MIN * MS_PER_S + 5*MS_PER_S*2, 
        SECONDS_PER_WINDOW * MS_PER_S
        )
    )

In [ ]:
ACTIVITY_CATEGORIES = ['Nonproduction', 'Input', 'Remove/Cut', 'Replace', 'Paste']

pipeline_activity_onehot = ColumnTransformer(
    transformers=[(
        'onehot_encode', 
        preprocessing.OneHotEncoder(
            categories=[ ACTIVITY_CATEGORIES ], 
            sparse=False, 
            handle_unknown='infrequent_if_exist'
            ),
        ["activity"]
    )],
    remainder='passthrough',
    verbose_feature_names_out=False
    )
pipeline_activity_onehot.fit(X_train)
original_categorical = X_train['activity']

X_train_dtypes = X_train.dtypes.to_dict()
X_train = pipeline_activity_onehot.transform(X_train)
X_train = pd.DataFrame(X_train, columns=pipeline_activity_onehot.get_feature_names_out())
X_train = pd.concat([X_train, original_categorical], axis=1)
X_train = X_train.astype(X_train_dtypes)

In [ ]:
for activity in ACTIVITY_CATEGORIES:

    X_train['burst_action_time_' + activity] = (
        X_train
        .assign(activity_x_event_time = lambda x: x['activity_' + activity] * x.action_time)
        .groupby(['id', 'burst_id'])
        ['activity_x_event_time']
        .transform('sum')
        ).astype(float)
    
X_train['burst_type'] = (
    X_train
    [['burst_action_time_' + activity for activity in ACTIVITY_CATEGORIES]]
    .idxmax(axis=1)
    )
X_train['burst_type'] = (
    X_train['burst_type']
    .str
    .replace("burst_action_time_", "", regex=True)
    )

In [ ]:
pipeline_burst_type_onehot = ColumnTransformer(
    transformers=[(
        'onehot_encode', 
        preprocessing.OneHotEncoder(
            categories=[ ACTIVITY_CATEGORIES ], 
            sparse=False, 
            handle_unknown='infrequent_if_exist'
            ),
        ["burst_type"]
    )],
    remainder='passthrough',
    verbose_feature_names_out=False
    )
pipeline_burst_type_onehot.fit(X_train)
original_categorical = X_train['burst_type']

X_train_dtypes = X_train.dtypes.to_dict()
X_train = pipeline_burst_type_onehot.transform(X_train)
X_train = pd.DataFrame(X_train, columns=pipeline_burst_type_onehot.get_feature_names_out())
X_train = pd.concat([X_train, original_categorical], axis=1)
X_train = X_train.astype(X_train_dtypes)

In [ ]:
for activity in ACTIVITY_CATEGORIES:

    X_train['is_new_burst_start_' + activity] = (
        X_train['is_new_burst_start'] * 
        X_train['burst_type_' + activity]
        )

In [ ]:
X_train = X_train[[
    "id",
    "event_id",
    "window_30s",
    "burst_id",
    "burst_type",
    "burst_type_Nonproduction",
    "burst_type_Input",
    "burst_type_Remove/Cut",
    "burst_type_Replace",
    "burst_type_Paste",
    "is_new_burst_start",
    "is_new_burst_start_Nonproduction",
    "is_new_burst_start_Input",
    "is_new_burst_start_Remove/Cut",
    "is_new_burst_start_Replace",
    "is_new_burst_start_Paste",
    "burst_time_start",
    "burst_time_end",
    "burst_duration",
    "word_count_delta_burst",

    "down_time",
    "up_time",	
    "action_time",	
    "activity",	
    "activity_Nonproduction",
    "activity_Input",
    "activity_Remove/Cut",
    "activity_Replace",
    "activity_Paste",
    "down_event",	
    "up_event",	
    "text_change",
    "is_thought_delimiting_punctuation",
    "cursor_position",	
    "word_count",

    "cursor_position_vs_max",
    "cursor_position_cummax",
    "has_cursor_position_moved_right",

    "word_count_lag1",
    "word_count_delta_event",

    "up_time_lag1",
    "preceding_pause_time",
    "preceding_pause_time_bucket",

    "burst_action_time_Nonproduction",
    "burst_action_time_Input",
    "burst_action_time_Remove/Cut",
    "burst_action_time_Replace",
    "burst_action_time_Paste"
    ]]


In [ ]:
vars_sum = (
    ['activity_' + x for x in ACTIVITY_CATEGORIES] 
    + ['is_new_burst_start'] 
    + ['is_new_burst_start_' + x for x in ACTIVITY_CATEGORIES]
    + ['word_count_delta_event']
    + ["is_thought_delimiting_punctuation"]
    + ["preceding_pause_time"]
    )

X_train_marginals_sum_wrt_time = (
    X_train
    .groupby('id')
    [vars_sum]
    .agg(sum)
    )
X_train_marginals_sum_wrt_time['delete_insert_ratio'] = (
    X_train_marginals_sum_wrt_time['activity_Remove/Cut'] / 
    X_train_marginals_sum_wrt_time['activity_Input'] 
    )

In [ ]:
X_train_marginals_central_tendency_wrt_time = (
    X_train
    .groupby('id')
    .agg(
        pause_time_p50 = ('preceding_pause_time', np.median),
        has_cursor_position_moved_right_mean = ('has_cursor_position_moved_right', 'mean'),
        burst_duration_mean = ('burst_duration', 'mean'),
        burst_duration_p50 = ('burst_duration', np.median),
        word_count_delta_burst_p50 = ('word_count_delta_burst', np.median),
        cursor_position_vs_max_avg = ('cursor_position_vs_max', 'mean')
        )
    )

In [ ]:
X_train_marginals_extremes_wrt_time = (
    X_train
    .groupby('id')
    .agg(
        pause_time_max=('preceding_pause_time', 'max'),
        # approximation to, next longest pause after first long planning pause
        pause_time_p99=('preceding_pause_time', lambda x: x.quantile(0.99)),
        burst_duration_max=('burst_duration', 'max'),
        total_time=('up_time', 'max')
        )
    )

In [ ]:
from scipy.stats import lognorm

pause_distr_summary_subjects = []

for X_train_subject in [x for _, x in X_train.groupby('id')]:

    shape, location, scale = lognorm.fit(X_train_subject['preceding_pause_time'].dropna())

    pause_distr_summary = pd.DataFrame({
        'pauses_lognorm_shape': [shape], 
        'pauses_lognorm_location': [location],
        'pauses_lognorm_scale': [scale]
        })
    pause_distr_summary.index = [X_train_subject['id'].iloc[0]]
    
    pause_distr_summary_subjects.append(pause_distr_summary)

X_train_marginals_distr_params_wrt_time = pd.concat(pause_distr_summary_subjects, axis=0)

In [ ]:
# word length offers a content quality measure
import re

is_input_new_character_addition = (
    (~ X_train['text_change'].str.contains('=>'))
    & (X_train['text_change'] != 'NoChange')
    )
tmp_df = \
df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
# collapse many records' text changes into a single cell
tmp_df = \
tmp_df.groupby('id').agg({'text_change': list}).reset_index()
# join a single string of the text changes in list elements
tmp_df['text_change'] = \
tmp_df['text_change'].apply(lambda x: ''.join(x))
# filter the string for phrases with text -- return a list of those elements
tmp_df['text_change'] = \
tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
# robustness check against recorded field
tmp_df['input_word_count'] = \
tmp_df['text_change'].apply(len)

# get lengths of words in 
tmp_df['input_word_length_mean'] = \
tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
tmp_df['input_word_length_max'] = \
tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
tmp_df['input_word_length_std'] = \
tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
tmp_df.drop(['text_change'], axis=1, inplace=True)

# ideal: _latest_ string since last space
    # that is, when cursor moves backward, not going to pick that up
# want accurate measure of word length
    # track inputs [text additions], subtractions [Remove/Cut, also recorded as 'q']

In [ ]:
X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_sum_wrt_time, 
    X_train_marginals_central_tendency_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_wrt_time, 
    X_train_marginals_extremes_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_wrt_time, 
    X_train_marginals_distr_params_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

In [ ]:
for var in vars_sum:

    if var == 'preceding_pause_time':
        var_out = 'pause_time_fraction'
    else:
        var_out = var + '_per_s'

    X_train_marginals_wrt_time[var_out] = (
        (X_train_marginals_wrt_time[var] / X_train_marginals_wrt_time['total_time'])
        )
    
    if 'per_s' in var_out:
        X_train_marginals_wrt_time[var_out] *= 1000

X_train_marginals_wrt_time = (
    X_train_marginals_wrt_time
    .assign(
        keystroke_speed = lambda x: (x.activity_Input + x['activity_Remove/Cut']) / x.total_time,
        words_per_thought_delimiting_punctuation = lambda x: x.word_count_delta_event / x.is_thought_delimiting_punctuation,
        )
    )

In [ ]:
X_train_marginals_wrt_time.head()

In [ ]:
X_train_marginals_wrt_time.isnull().sum()

In [ ]:
X_train_by_window = (
    X_train
    .groupby(['id', 'window_30s'])
    [vars_sum + ['cursor_position_vs_max']]
    .agg(sum)
    .astype(float)
    .reset_index(drop=False)
    )

X_train_by_window['cursor_position_vs_max'] = (
    X_train_by_window['cursor_position_vs_max'] / 
    X_train_by_window[['activity_' + x for x in ACTIVITY_CATEGORIES]].sum(axis=1)
    )

X_train_by_window['delete_insert_ratio'] = (
    X_train_by_window['activity_Remove/Cut'] / 
    X_train_by_window['activity_Input'] 
    )

X_train_by_window['window_30s_idx'] = X_train_by_window.index

# for variability measure more comparable between writers,
# de-mean by writer. 
# Ex: higher-throughput writer incurs higher stddev, because values have higher abs value
time_rate_normalizers = [
    x
    for x in X_train_marginals_wrt_time.columns
    if 'per_s' in x 
    ]
# join method allows for merge on single index column
X_train_by_window = X_train_by_window.join(
    X_train_marginals_wrt_time[time_rate_normalizers],
    on='id',
    how='left'
)
for denom in time_rate_normalizers:
    level = denom.replace("_per_s", "")
    X_train_by_window[level] = (
        X_train_by_window[level] / 
        (X_train_by_window[denom].replace(0, None) * 30)
        )
    X_train_by_window[level] = X_train_by_window[level].fillna(1)
    
X_train_by_window = X_train_by_window.drop(columns=time_rate_normalizers)

X_train_by_window['preceding_pause_time'] = (
    X_train_by_window['preceding_pause_time'] / (1000 * 30)
    )

In [ ]:
X_train_windows_variation = (
    X_train_by_window
    .drop(columns=['window_30s', 'window_30s_idx'])
    .groupby(['id'])
    .agg(np.std)
    )

X_train_windows_variation.columns = [
    x + "_stddev"
    for x in X_train_windows_variation.columns
    ]

In [ ]:
X_train_windows_variation.head()

In [ ]:
X_train_transform = pd.merge(
    X_train_marginals_wrt_time,
    X_train_windows_variation,
    how='left',
    left_index=True,
    right_index=True
    )

In [ ]:
X_train_transform

In [ ]:
X_train_transform.isnull().mean()

In [ ]:
CURSOR_POSITION_VS_MAX_STDDEV_P50 = 246.6

X_train_transform['cursor_position_vs_max_stddev'] = (
    X_train_transform['cursor_position_vs_max_stddev'].fillna(CURSOR_POSITION_VS_MAX_STDDEV_P50)
)

In [ ]:
(
    X_train_transform
    .drop(columns='delete_insert_ratio_stddev')
    .to_pickle("./data/processed/X_train.pkl")
)